In [19]:
import matplotlib
matplotlib.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()

import mne
mne.viz.set_browser_backend('qt')
mne.set_log_level('WARNING')

import os
import numpy as np
from prep_function import (annotate_dynamic_p2p, base_filtering,
                           load_muse_data, remove_ica_comp_and_plot, run_ica, plot_and_save_psd)
from prep_output import export_all_data_to_csv, extract_frequency_bands

In [20]:
csv_path = r"mindMonitor_2025-05-21--23-26-36.csv"
output_dir = "final_outputs"
os.makedirs(output_dir, exist_ok=True)

## Step 1: loading and interpolating nan values

In [21]:
raw = load_muse_data(csv_path)

Percentage of NaN values interpolated in EEG data: 6.39%


In [22]:
raw.plot()

## Step 2: filtering 

In [23]:
base_filtering(raw)
print(raw.info)

<Info | 9 non-empty values
 bads: []
 ch_names: AF7, AF8, TP9, TP10
 chs: 4 EEG
 custom_ref_applied: True
 dig: 7 items (3 Cardinal, 4 EEG)
 highpass: 1.0 Hz
 lowpass: 40.0 Hz
 meas_date: unspecified
 nchan: 4
 projs: []
 sfreq: 256.0 Hz
>


In [24]:
raw.plot()

## Step 3: auto-annotating artifacts

In [25]:
raw = annotate_dynamic_p2p(raw, n_mads=8, win_sec=0.05, step_sec=0.01)

/Users/morzadanaveh/Documents/development/Python/Hackaton/Muse-S-EEG-prep/prep_function.py:79: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(raw.annotations + new_ann)


In [26]:
raw.plot()

<div style="
     border: 2px solid #d9534f;
     background-color: #f2dede;
     color: #a94442;
     padding: 10px 15px;
     border-radius: 4px;
     margin: 10px 0;
">
<strong>⚠️ IMPORTANT:</strong> To experiment with new parameters:
  <ol>
    <li>Run the cell below to reset your annotations.</li>
    <li>Update the parameters in the function <em>annotate_dynamic_p2p</em> above.</li>
    <li>Re-run the cell above with your new settings.</li>
  </ol>
</div>

In [27]:
# NOTICE: this line resets the annotations.
_ = raw.set_annotations(raw.annotations[np.array(raw.annotations.description) != 'BAD_dynamic'])

## Step 4: performing ICA

Note: The next cell will pop out some plots of the ICA components. Inspect them and decide which ones to remove (if any).

In [28]:
# Run ICA
ica = run_ica(raw)

/Users/morzadanaveh/Documents/development/Python/Hackaton/Muse-S-EEG-prep/prep_function.py:149: RuntimeWarning: Using n_components=4 (resulting in n_components_=4) may lead to an unstable mixing matrix estimation because the ratio between the largest (3.3) and smallest (9.7e-31) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 3
  ica.fit(raw, reject_by_annotation=True)


In [29]:
comps_to_remove = [1] # add the component indices you want to remove

In [30]:
if comps_to_remove:
    after_ica = remove_ica_comp_and_plot(ica, raw, comps_to_remove)

## Step 5: extract frequency bands per channel

In [31]:
per_channel_raws = extract_frequency_bands(raw)

In [32]:
per_channel_raws['TP9'].plot()

## Step 6: plot PSD

In [33]:
plot_and_save_psd(raw, output_dir, min_duration=1.0)

## Step 6: save the data

In [34]:
for curr_chan, curr_raw in per_channel_raws.items():
    curr_raw.save(os.path.join(output_dir, f"{curr_chan}_decomposed_raw.fif"), overwrite=True)

In [35]:
csv_path = f'{output_dir}/all_channels_with_bands.csv'
export_all_data_to_csv(per_channel_raws, csv_path)

Saved combined data to final_outputs/all_channels_with_bands.csv
